# 25-最邻近插值（ Nearest-neighbor Interpolation ）

使用最邻近插值将图像放大$1.5$倍

最近邻插值在图像放大时补充的像素取最临近的像素的值。由于方法简单，所以处理速度很快，但是放大图像画质劣化明显。

使用下面的公式放大图像 $I'$为放大后图像，$I$为放大前图像，$a$为放大率，方括号是四舍五入取整操作：

$$ I'(x,y) = I([\frac{x}{a}], [\frac{y}{a}]) $$

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [4]:
img = cv2.imread('../picture/chan.jpg').astype(np.float)

In [5]:
def interpolation(img, ay, ax):
    
    h, w, c = img.shape
    
    ah = int(ay * h)
    aw = int(ax * w)
    
    y = np.arange(ah).repeat(aw).reshape(aw, -1)
    x = np.tile(np.arange(aw), (ah, 1))
    y = np.round(y / ay).astype(np.int)
    x = np.round(x / ax).astype(np.int)
    
    out = img[y, x]
    out = out.astype(np.uint8)

    return out

In [6]:
img_inter = interpolation(img, ay=1.5,ax=1.5)

In [7]:
cv2.imwrite('../picture/chan_result25_nearest_neighbor_interpolation.jpg', img_inter)
cv2.namedWindow("result", 0);
cv2.resizeWindow("result", (600, 600));
cv2.imshow("result", img_inter)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
img.shape,img_inter.shape

((540, 540, 3), (810, 810, 3))

# 26-双线性插值（ Bilinear Interpolation ）

使用双线性插值将图像放大$1.5$倍

双线性插值考察$4$邻域的像素点，并根据距离设置权值。虽然计算量增大使得处理时间变长，但是可以有效抑制画质劣化。

1. 放大后图像的座标$(x',y')$除以放大率$a$，可以得到对应原图像的座标$(\lfloor \frac{x'}{a}\rfloor , \lfloor \frac{y'}{a}\rfloor)$。

2. 求原图像的座标$(\lfloor \frac{x'}{a}\rfloor , \lfloor \frac{y'}{a}\rfloor)$周围$4$邻域的座标$I(x,y)$，$I(x+1,y)$，$I(x,y+1)$，$I(x+1, y+1)$：

3. 分别求这4个点与$(\frac{x'}{a}, \frac{y'}{a})$的距离，根据距离设置权重：$w = \frac{d}{\sum\ d}$

4. 根据下式求得放大后图像$(x',y')$处的像素值： $$ d_x = \frac{x'}{a} - x\ d_y = \frac{y'}{a} - y\ I'(x',y') = (1-d_x)\ (1-d_y)\ I(x,y) + d_x\ (1-d_y)\ I(x+1,y) + (1-d_x)\ d_y\ I(x,y+1) + d_x\ d_y\ I(x+1,y+1) $$ 

In [9]:
def bil(img, ax=1.5, ay=1.5):
    
    h, w, c = img.shape
    
    ah = int(ay * h)
    aw = int(ax * w)
    
    y = np.arange(ah).repeat(aw).reshape(aw, -1)
    x = np.tile(np.arange(aw), (ah, 1))
    
    y = y / ay
    x = x / ax
    
    ix = np.floor(x).astype(np.int)
    iy = np.floor(y).astype(np.int)
    
    ix = np.minimum(ix, w-2)
    iy = np.minimum(iy, h-2)
    
    dx = x - ix
    dy = y - iy

    dx = np.repeat(np.expand_dims(dx, axis=-1), 3, axis=-1)
    dy = np.repeat(np.expand_dims(dy, axis=-1), 3, axis=-1)

    out = (1-dx) * (1-dy) * img[iy, ix] + dx * (1 - dy) * img[iy, ix+1] + (1 - dx) * dy * img[iy+1, ix] + dx * dy * img[iy+1, ix+1]

    out = np.clip(out, 0, 255)
    out = out.astype(np.uint8)

    return out

In [10]:
img_bil = bil(img, ax=1.5, ay=1.5)

In [11]:
cv2.imwrite('../picture/chan_result26_Bilinear Interpolation.jpg', img_bil)
cv2.namedWindow("result", 0);
cv2.resizeWindow("result", (600, 600));
cv2.imshow("result", img_bil)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
img_bil.shape

(810, 810, 3)

# 27-双三次插值（ Bicubic Interpolation ）

使用双三次插值将图像放大$1.5$倍

双三次插值是双线性插值的扩展，使用邻域$16$像素进行插值。

各自像素间的距离由下式决定： $$ \begin{align*} d_{x_1} = |\frac{x'}{a\ x} - (x-1)|\quad d_{x_2} = |\frac{x'}{a\ x}- x| \quad d_{x_3} = |\frac{x'}{a\ x}- (x+1)|\quad d_{x_4} = |\frac{x'}{a\ x} - (x+2)|\ d_{y_1} = |\frac{x'}{a\ y} - (y-1)|\quad d_{y_2} = |\frac{x'}{a\ y} - y| \quad d_{y_3} = |\frac{x'}{a\ y} - (y+1)| \quad d_{y_4} = |\frac{x'}{a\ y} - (y+2)| \end{align*} $$ 权重由基于距离的函数取得。$a$在大部分时候取$-1$。大体上说，图中蓝色像素的距离$|t|\leq 1$，绿色像素的距离$1<|t|\leq 2$： $$ h(t)= \begin{cases} (a+2)\ |t|^3 - (a+3)\ |t|^2 + 1 &\text{when}\quad |t|\leq 1 \ a\ |t|^3 - 5\ a\ |t|^2 + 8\ a\ |t| - 4\ a&\text{when}\quad 1<|t|\leq 2\ 0&\text{else} \end{cases} $$ 利用上面得到的权重，通过下面的式子扩大图像。将每个像素与权重的乘积之和除以权重的和。 $$ I'(x', y')=\frac{1}{\sum\limits_{j=1}^4\ \sum\limits_{i=1}^4\ h(d_{xi})\ h(d_{yj})}\ \sum\limits_{j=1}^4\ \sum\limits_{i=1}^4\ I(x+i-2,y+j-2)\ h(d_{xi})\ h(d_{yj}) $$

In [17]:
def bc_interpolate(img, ax=1.5, ay=1.5):
    
    h, w, c = img.shape

    ah = int(ay * h)
    aw = int(ax * w)

    y = np.arange(ah).repeat(aw).reshape(aw, -1)
    x = np.tile(np.arange(aw), (ah, 1))
    y = (y / ay)
    x = (x / ax)

    ix = np.floor(x).astype(np.int)
    iy = np.floor(y).astype(np.int)

    ix = np.minimum(ix, w-1)
    iy = np.minimum(iy, h-1)

    dx2 = x - ix
    dy2 = y - iy
    dx1 = dx2 + 1
    dy1 = dy2 + 1
    dx3 = 1 - dx2
    dy3 = 1 - dy2
    dx4 = 1 + dx3
    dy4 = 1 + dy3

    dxs = [dx1, dx2, dx3, dx4]
    dys = [dy1, dy2, dy3, dy4]

    def weight(t):
        a = -1.
        at = np.abs(t)
        w = np.zeros_like(t)
        ind = np.where(at <= 1)
        w[ind] = ((a+2) * np.power(at, 3) - (a+3) * np.power(at, 2) + 1)[ind]
        ind = np.where((at > 1) & (at <= 2))
        w[ind] = (a*np.power(at, 3) - 5*a*np.power(at, 2) + 8*a*at - 4*a)[ind]
        return w

    w_sum = np.zeros((ah, aw, c), dtype=np.float32)
    out = np.zeros((ah, aw, c), dtype=np.float32)

    for j in range(-1, 3):
            for i in range(-1, 3):
                ind_x = np.minimum(np.maximum(ix + i, 0), w-1)
                ind_y = np.minimum(np.maximum(iy + j, 0), h-1)

                wx = weight(dxs[i+1])
                wy = weight(dys[j+1])
                wx = np.repeat(np.expand_dims(wx, axis=-1), 3, axis=-1)
                wy = np.repeat(np.expand_dims(wy, axis=-1), 3, axis=-1)

                w_sum += wx * wy
                out += wx * wy * img[ind_y, ind_x]

    out /= w_sum
    out = np.clip(out, 0, 255)
    out = out.astype(np.uint8)

    return out

In [18]:
img_bc = bc_interpolate(img, ax=1.5, ay=1.5)